In [1]:
#importing some useful packages

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
from PIL import Image
import keras
import math
import random
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam, RMSprop


Using TensorFlow backend.


In [2]:
import pandas as pd
data = pd.read_csv('driving_log.csv', header = None)
data.columns = ["center_images","left_images","right_images","steering","brake","throttle","speed"]
len(data)

27007

In [3]:
Y_train = np.copy(data['steering'])  
len(Y_train)

27007

In [4]:
X_train = np.copy(data['center_images']+':'+data['left_images']+':'+data['right_images'])
len(X_train)

27007

In [5]:
X_train = np.copy(data['center_images']+':'+data['left_images']+':'+data['right_images'])
Y_train = np.copy(data['steering'])  

In [6]:
len(X_train)

27007

In [7]:
X_train, X_validation, Y_train, Y_validation = train_test_split(
            X_train,
            Y_train,
            test_size=0.2,
            random_state=0)
len(X_train)

21605

In [8]:
img_rows = 40
img_cols = 80
ch = 3

In [9]:
len(X_validation)

5402

In [10]:
def load_image(imagepath):
    oriimage = cv2.imread(imagepath, 1)
    # do any preprocessing... resize, reshape, etc. here...
    newx,newy = int(oriimage.shape[1]/4),int(oriimage.shape[0]/4) #new size (w,h)
    newimage = cv2.resize(oriimage,(newx,newy))
    return newimage

In [11]:
counter = 0
batch_size = 32
nb_epoch = 2

train_images = np.zeros((32,40,80,3))
train_steering = np.zeros(32)
validation_images = np.zeros((8,40,80,3))
validation_steering = np.zeros(8)

#Change to float32, so it can be used in model.fit_generator
#train_images = train_images.astype(np.float32)
#train_steering = train_steering.astype(np.float32)

In [12]:
"""
Steering angle prediction model
"""
import os
import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D



 

In [13]:
# our image generator.
def batchgen(X, Y):
    while 1:
        for i in range(len(X)):
            y = Y[i]
            if y < -0.01:
                chance = random.random()
                if chance > 0.75:
                    imagepath = X[i].split(':')[1]
                    y *= 3.0
                else:
                    if chance > 0.5:
                        imagepath = X[i].split(':')[1]
                        y *= 2.0
                    else:
                        if chance > 0.25:
                            imagepath = X[i].split(':')[0]
                            y *= 1.5
                        else:
                            imagepath = X[i].split(':')[0]
            else:
                if y > 0.01:
                    chance = random.random()
                    if chance > 0.75:
                        imagepath = X[i].split(':')[2]
                        y *= 3.0
                    else:
                        if chance > 0.5:
                            imagepath = X[i].split(':')[2]
                            y *= 2.0
                        else:
                            if chance > 0.25:
                                imagepath = X[i].split(':')[0]
                                y *= 1.5
                            else:
                                imagepath = X[i].split(':')[0]
                else:
                    imagepath = X[i].split(':')[0]
            #print(imagepath)
            image = load_image(imagepath)
            y = np.array([[y]])
            image = image.reshape(1, img_rows, img_cols, ch)
            yield image, y

In [14]:
def get_model(time_len=1):
    row, col, ch = 40, 80, 3  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(row, col, ch),
            output_shape=(row, col, ch)))
    
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))
    
  
    model.compile(optimizer=Adam(), loss='mse')

    return model

In [ ]:
model = get_model()

In [ ]:
history = model.fit_generator(batchgen(X_train, Y_train), samples_per_epoch = 21605, nb_epoch = 8,
                    verbose=1, callbacks=[], validation_data=batchgen(X_validation, Y_validation),
                    nb_val_samples=8, class_weight=None,
                     pickle_safe=False)

Epoch 1/8
21605/21605 [==============================] - 257s - loss: 0.4199 - val_loss: 0.0873
Epoch 2/8
21605/21605 [==============================] - 397s - loss: 0.4810 - val_loss: 0.9823
Epoch 3/8
21605/21605 [==============================] - 657s - loss: 0.4032 - val_loss: 0.2192
Epoch 4/8
 8421/21605 [==========>...................] - ETA: 405s - loss: 0.3838

In [ ]:
print("Saving model weights and configuration file.")

if not os.path.exists("./outputs/steering_model"):
        os.makedirs("./outputs/steering_model")

model.save_weights("./outputs/steering_model/steering_angle.keras", True)
with open('./outputs/steering_model/steering_angle.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)